In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/02 18:24:38 WARN Utils: Your hostname, queen resolves to a loopback address: 127.0.1.1; using 192.168.1.124 instead (on interface wlo1)
24/03/02 18:24:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/02 18:24:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/02 18:24:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE 
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY 
    1, 2
```

In [7]:
df_green.rdd.take(2)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 14, 32), lpep_dropoff_datetime=datetime.datetime(2020, 1, 29, 14, 37, 24), store_and_fwd_flag='N', RatecodeID=1, PULocationID=97, DOLocationID=97, passenger_count=1, trip_distance=0.81, fare_amount=5.5, extra=0.0, mta_tax=0.5, tip_amount=1.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=7.3, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 24, 4, 36, 44), lpep_dropoff_datetime=datetime.datetime(2020, 1, 24, 4, 46, 50), store_and_fwd_flag='N', RatecodeID=5, PULocationID=7, DOLocationID=141, passenger_count=1, trip_distance=3.02, fare_amount=40.0, extra=0.0, mta_tax=0.0, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.0, total_amount=42.75, payment_type=1, trip_type=2, congestion_surcharge=2.75)]

In [8]:
df_green.take(2)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 14, 32), lpep_dropoff_datetime=datetime.datetime(2020, 1, 29, 14, 37, 24), store_and_fwd_flag='N', RatecodeID=1, PULocationID=97, DOLocationID=97, passenger_count=1, trip_distance=0.81, fare_amount=5.5, extra=0.0, mta_tax=0.5, tip_amount=1.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=7.3, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 24, 4, 36, 44), lpep_dropoff_datetime=datetime.datetime(2020, 1, 24, 4, 46, 50), store_and_fwd_flag='N', RatecodeID=5, PULocationID=7, DOLocationID=141, passenger_count=1, trip_distance=3.02, fare_amount=40.0, extra=0.0, mta_tax=0.0, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.0, total_amount=42.75, payment_type=1, trip_type=2, congestion_surcharge=2.75)]

In [9]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [10]:
rdd.take(2)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 14, 32), PULocationID=97, total_amount=7.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 24, 4, 36, 44), PULocationID=7, total_amount=42.75)]

## Implementing Filter

In [14]:
from datetime import datetime

In [15]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [16]:
rdd.filter(filter_outliers).take(2)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 14, 32), PULocationID=97, total_amount=7.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 24, 4, 36, 44), PULocationID=7, total_amount=42.75)]

## Group By

### Prepare for grouping

In [17]:
rows = rdd.take(10)
row = rows[0]
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 14, 32), PULocationID=97, total_amount=7.3)

In [18]:
row.lpep_pickup_datetime

datetime.datetime(2020, 1, 29, 14, 32)

In [19]:
row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)

datetime.datetime(2020, 1, 29, 14, 0)

In [20]:
def prepare_for_grouping(row):
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)

    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [21]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .take(2)

[((datetime.datetime(2020, 1, 29, 14, 0), 97), (7.3, 1)),
 ((datetime.datetime(2020, 1, 24, 4, 0), 7), (42.75, 1))]

### Reduce by key

In [22]:
def calculate_revenue(left_value, right_value):
    l_amount, l_count = left_value
    r_amount, r_count = right_value
    
    return (l_amount+r_amount, l_count+r_count)

In [23]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .take(2)

[((datetime.datetime(2020, 1, 26, 11, 0), 166), (373.69000000000005, 23)),
 ((datetime.datetime(2020, 1, 30, 17, 0), 82), (503.90000000000015, 37))]

### Unnest/unwrap the tuples

In [27]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(lambda row: (row[0][0], row[0][1], row[1][0], row[1][1])) \
    .take(2)

[(datetime.datetime(2020, 1, 26, 11, 0), 166, 373.69000000000005, 23),
 (datetime.datetime(2020, 1, 30, 17, 0), 82, 503.90000000000015, 37)]

In [28]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(lambda row: (row[0][0], row[0][1], row[1][0], row[1][1])) \
    .toDF() \
    .take(2)

[Row(_1=datetime.datetime(2020, 1, 26, 11, 0), _2=166, _3=373.69000000000005, _4=23),
 Row(_1=datetime.datetime(2020, 1, 30, 17, 0), _2=82, _3=503.90000000000015, _4=37)]

In [29]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(lambda row: (row[0][0], row[0][1], row[1][0], row[1][1])) \
    .toDF() \
    .show(2)

+-------------------+---+------------------+---+
|                 _1| _2|                _3| _4|
+-------------------+---+------------------+---+
|2020-01-26 11:00:00|166|373.69000000000005| 23|
|2020-01-30 17:00:00| 82|503.90000000000015| 37|
+-------------------+---+------------------+---+
only showing top 2 rows



In [30]:
from collections import namedtuple

In [31]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [34]:
df_result= rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF()

In [35]:
df_result.show(2)

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-26 11:00:00| 166|373.69000000000005|   23|
|2020-01-30 17:00:00|  82|503.90000000000015|   37|
+-------------------+----+------------------+-----+
only showing top 2 rows



### Set schema

In [36]:
df_result.schema

StructType([StructField('hour', TimestampType(), True), StructField('zone', LongType(), True), StructField('revenue', DoubleType(), True), StructField('count', LongType(), True)])

In [37]:
from pyspark.sql import types

In [38]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [39]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [40]:
df_result.write.parquet('tmp/green-revenue')

24/03/02 19:03:37 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/02 19:03:37 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/02 19:03:37 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/03/02 19:03:37 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/03/02 19:03:37 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/03/02 19:03:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/03/02 19:03:39 WARN MemoryManager: Total allocation exceeds 95.